In [396]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.functions import regexp_replace

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [397]:
# read data from the silver tables
champions = spark.table("NBA.sl_Champions")
draft= spark.table("NBA.sl_Draft_picks")

In [398]:
from pyspark.sql.functions import when, col, concat, lit

champions = champions.withColumn(    
    "Contador_Campeoes",
    lit(1)
)


In [399]:
champions.show(3000)

+----+------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
|Year|League|            Champion|           Runner_Up|      Finals_MVP|              Points|            Rebounds|             Assists|          Win_Shares|                Team|Contador_Campeoes|
+----+------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
|2008|   NBA|      Boston Celtics|  Los Angeles Lakers|       P. Pierce|     K. Bryant (633)|    K. Garnett (274)|      R. Rondo (172)|    K. Garnett (4.1)|      Boston Celtics|                1|
|1986|   NBA|      Boston Celtics|     Houston Rockets|         L. Bird|   H. Olajuwon (537)|   H. Olajuwon (236)|    M. Johnson (211)|       L. Bird (4.2)|      Boston Celtics|                1|
|1984|   NBA|      B

In [400]:
from pyspark.sql.functions import substring, avg, sum

champions_quantity = champions \
    .groupBy("Champion") \
    .agg(
        sum(champions.Contador_Campeoes).cast('INT').alias("Contador_Campeoes"))

In [401]:
champions_quantity.show(3000)

+--------------------+-----------------+
|            Champion|Contador_Campeoes|
+--------------------+-----------------+
|      Boston Celtics|               17|
|    Dallas Mavericks|                1|
|     New York Knicks|                2|
|                null|               11|
|  Los Angeles Lakers|               16|
|Golden State Warr...|                5|
|     Detroit Pistons|                3|
|       Chicago Bulls|                6|
|     Milwaukee Bucks|                2|
| Cleveland Cavaliers|                1|
|  Washington Wizards|                1|
|       Atlanta Hawks|                1|
|Portland Trail Bl...|                1|
|     Toronto Raptors|                1|
|  Philadelphia 76ers|                2|
|   San Antonio Spurs|                5|
|Oklahoma City Thu...|                1|
|    Sacramento Kings|                1|
|     Houston Rockets|                2|
|          Miami Heat|                3|
+--------------------+-----------------+



In [402]:
from pyspark.sql.functions import substring, avg, sum

runner_up_quantity = champions \
    .groupBy("Runner_Up") \
    .agg(
        sum(champions.Contador_Campeoes).cast('INT').alias("Contador_Campeoes"))

In [403]:
runner_up_quantity.show()

+--------------------+-----------------+
|           Runner_Up|Contador_Campeoes|
+--------------------+-----------------+
|        Phoenix Suns|                3|
|      Boston Celtics|                4|
|    Dallas Mavericks|                1|
|       Brooklyn Nets|                2|
|     New York Knicks|                6|
|                null|               11|
|  Los Angeles Lakers|               15|
|  Syracuse Nationals|                2|
|Golden State Warr...|                4|
|       Orlando Magic|                2|
|     Detroit Pistons|                3|
|     Milwaukee Bucks|                1|
| Cleveland Cavaliers|                4|
|      Indiana Pacers|                1|
|  Washington Wizards|                2|
|       Atlanta Hawks|                3|
|           Utah Jazz|                2|
|  Philadelphia 76ers|                4|
|Portland Trail Bl...|                2|
|   San Antonio Spurs|                1|
+--------------------+-----------------+
only showing top

In [404]:
champions= champions.drop('Contador_Campeoes')
champions= champions.drop('Points')
champions= champions.drop('Rebounds')
champions= champions.drop('Assists')
champions= champions.drop('Win_Shares')
champions= champions.drop('Finals_MVP')
champions_quantity = champions_quantity.withColumnRenamed("Champion","Champions")
runner_up_quantity = runner_up_quantity.withColumnRenamed("Runner_Up","Runner_Ups")



In [405]:
champions_quantity.show(3000)

+--------------------+-----------------+
|           Champions|Contador_Campeoes|
+--------------------+-----------------+
|      Boston Celtics|               17|
|    Dallas Mavericks|                1|
|     New York Knicks|                2|
|                null|               11|
|  Los Angeles Lakers|               16|
|Golden State Warr...|                5|
|     Detroit Pistons|                3|
|       Chicago Bulls|                6|
|     Milwaukee Bucks|                2|
| Cleveland Cavaliers|                1|
|  Washington Wizards|                1|
|       Atlanta Hawks|                1|
|Portland Trail Bl...|                1|
|     Toronto Raptors|                1|
|  Philadelphia 76ers|                2|
|   San Antonio Spurs|                5|
|Oklahoma City Thu...|                1|
|    Sacramento Kings|                1|
|     Houston Rockets|                2|
|          Miami Heat|                3|
+--------------------+-----------------+



In [406]:
champions.show()

+----+------+------------------+--------------------+------------------+
|Year|League|          Champion|           Runner_Up|              Team|
+----+------+------------------+--------------------+------------------+
|2008|   NBA|    Boston Celtics|  Los Angeles Lakers|    Boston Celtics|
|1986|   NBA|    Boston Celtics|     Houston Rockets|    Boston Celtics|
|1984|   NBA|    Boston Celtics|  Los Angeles Lakers|    Boston Celtics|
|1981|   NBA|    Boston Celtics|     Houston Rockets|    Boston Celtics|
|1976|   NBA|    Boston Celtics|        Phoenix Suns|    Boston Celtics|
|1974|   NBA|    Boston Celtics|     Milwaukee Bucks|    Boston Celtics|
|1969|   NBA|    Boston Celtics|  Los Angeles Lakers|    Boston Celtics|
|1968|   NBA|    Boston Celtics|  Los Angeles Lakers|    Boston Celtics|
|1966|   NBA|    Boston Celtics|  Los Angeles Lakers|    Boston Celtics|
|1965|   NBA|    Boston Celtics|  Los Angeles Lakers|    Boston Celtics|
|1964|   NBA|    Boston Celtics|Golden State Warr..

In [407]:

cond = [champions.Champion == champions_quantity.Champions]
champions=champions.join(champions_quantity,cond, 'outer')

In [408]:
runner_up_quantity = runner_up_quantity.withColumnRenamed("Contador_Campeoes","Contador_Runner_Ups")


In [409]:
runner_up_quantity.show()

+--------------------+-------------------+
|          Runner_Ups|Contador_Runner_Ups|
+--------------------+-------------------+
|        Phoenix Suns|                  3|
|      Boston Celtics|                  4|
|    Dallas Mavericks|                  1|
|       Brooklyn Nets|                  2|
|     New York Knicks|                  6|
|                null|                 11|
|  Los Angeles Lakers|                 15|
|  Syracuse Nationals|                  2|
|Golden State Warr...|                  4|
|       Orlando Magic|                  2|
|     Detroit Pistons|                  3|
|     Milwaukee Bucks|                  1|
| Cleveland Cavaliers|                  4|
|      Indiana Pacers|                  1|
|  Washington Wizards|                  2|
|       Atlanta Hawks|                  3|
|           Utah Jazz|                  2|
|  Philadelphia 76ers|                  4|
|Portland Trail Bl...|                  2|
|   San Antonio Spurs|                  1|
+----------

In [410]:
champions.show()

+----+------+----------------+--------------------+----------------+----------------+-----------------+
|Year|League|        Champion|           Runner_Up|            Team|       Champions|Contador_Campeoes|
+----+------+----------------+--------------------+----------------+----------------+-----------------+
|2008|   NBA|  Boston Celtics|  Los Angeles Lakers|  Boston Celtics|  Boston Celtics|               17|
|1986|   NBA|  Boston Celtics|     Houston Rockets|  Boston Celtics|  Boston Celtics|               17|
|1984|   NBA|  Boston Celtics|  Los Angeles Lakers|  Boston Celtics|  Boston Celtics|               17|
|1981|   NBA|  Boston Celtics|     Houston Rockets|  Boston Celtics|  Boston Celtics|               17|
|1976|   NBA|  Boston Celtics|        Phoenix Suns|  Boston Celtics|  Boston Celtics|               17|
|1974|   NBA|  Boston Celtics|     Milwaukee Bucks|  Boston Celtics|  Boston Celtics|               17|
|1969|   NBA|  Boston Celtics|  Los Angeles Lakers|  Boston Celt

In [411]:
cond = [champions.Team == runner_up_quantity.Runner_Ups]
champions=champions.join(runner_up_quantity,cond, 'outer')

In [412]:
champions= champions.drop('Champions')
champions.show(3000)

+----+------+--------------------+--------------------+--------------------+-----------------+--------------------+-------------------+
|Year|League|            Champion|           Runner_Up|                Team|Contador_Campeoes|          Runner_Ups|Contador_Runner_Ups|
+----+------+--------------------+--------------------+--------------------+-----------------+--------------------+-------------------+
|null|  null|                null|                null|        Phoenix Suns|             null|        Phoenix Suns|                  3|
|2008|   NBA|      Boston Celtics|  Los Angeles Lakers|      Boston Celtics|               17|      Boston Celtics|                  4|
|1986|   NBA|      Boston Celtics|     Houston Rockets|      Boston Celtics|               17|      Boston Celtics|                  4|
|1984|   NBA|      Boston Celtics|  Los Angeles Lakers|      Boston Celtics|               17|      Boston Celtics|                  4|
|1981|   NBA|      Boston Celtics|     Houston R

In [413]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, HiveContext
from pyspark.sql import functions as F


champions = champions.withColumn("Contador_Campeoes", F.when(F.col("Contador_Campeoes").isNull(), 0).otherwise(F.col("Contador_Campeoes")))

In [414]:
champions.show(40)

+----+------+------------------+--------------------+--------------------+-----------------+------------------+-------------------+
|Year|League|          Champion|           Runner_Up|                Team|Contador_Campeoes|        Runner_Ups|Contador_Runner_Ups|
+----+------+------------------+--------------------+--------------------+-----------------+------------------+-------------------+
|null|  null|              null|                null|        Phoenix Suns|                0|      Phoenix Suns|                  3|
|2008|   NBA|    Boston Celtics|  Los Angeles Lakers|      Boston Celtics|               17|    Boston Celtics|                  4|
|1986|   NBA|    Boston Celtics|     Houston Rockets|      Boston Celtics|               17|    Boston Celtics|                  4|
|1984|   NBA|    Boston Celtics|  Los Angeles Lakers|      Boston Celtics|               17|    Boston Celtics|                  4|
|1981|   NBA|    Boston Celtics|     Houston Rockets|      Boston Celtics|  

In [415]:
draft = draft.filter((draft.Pk <= 60) )

In [416]:
from pyspark.sql.functions import substring, avg, sum

draft_avg= draft \
.groupBy("Team")\
.agg(
    avg(draft.Pk).alias("Pk_avg"),
)

In [417]:
draft.show(3000)

+--------------------+-------+--------+--------+---+--------------------+---+-----+--------------+-----+-----+
|              Player|All_NBA|All_Star|Draft_Yr| Pk|                Team|Yrs|Games|Minutes_Played|  PTS|  AST|
+--------------------+-------+--------+--------+---+--------------------+---+-----+--------------+-----+-----+
|       James Johnson|      0|       0|    2009| 16|       Chicago Bulls|  7|  398|          7270| 2594|  590|
|          Taj Gibson|      0|       0|    2009| 26|       Chicago Bulls|  7|  507|         12668| 4641|  504|
|      Kevin Seraphin|      0|       0|    2010| 17|       Chicago Bulls|  6|  374|          5871| 2284|  223|
|         Norris Cole|      0|       0|    2011| 28|       Chicago Bulls|  5|  347|          7893| 2463|  967|
|        Jimmy Butler|      0|       0|    2011| 30|       Chicago Bulls|  5|  323|         10071| 4392|  837|
|         Malcolm Lee|      0|       0|    2011| 43|       Chicago Bulls|  3|   36|           534|  140|   51|
|

In [418]:
draft= draft.drop('Pk')

In [419]:
draft.show(100)

+-----------------+-------+--------+--------+-------------+---+-----+--------------+-----+----+
|           Player|All_NBA|All_Star|Draft_Yr|         Team|Yrs|Games|Minutes_Played|  PTS| AST|
+-----------------+-------+--------+--------+-------------+---+-----+--------------+-----+----+
|    James Johnson|      0|       0|    2009|Chicago Bulls|  7|  398|          7270| 2594| 590|
|       Taj Gibson|      0|       0|    2009|Chicago Bulls|  7|  507|         12668| 4641| 504|
|   Kevin Seraphin|      0|       0|    2010|Chicago Bulls|  6|  374|          5871| 2284| 223|
|      Norris Cole|      0|       0|    2011|Chicago Bulls|  5|  347|          7893| 2463| 967|
|     Jimmy Butler|      0|       0|    2011|Chicago Bulls|  5|  323|         10071| 4392| 837|
|      Malcolm Lee|      0|       0|    2011|Chicago Bulls|  3|   36|           534|  140|  51|
|   Marquis Teague|      0|       0|    2012|Chicago Bulls|  2|   88|           835|  206| 121|
|       Tony Snell|      0|       0|    

In [420]:
draft_avg.show(3000)

+--------------------+------------------+
|                Team|            Pk_avg|
+--------------------+------------------+
|        Phoenix Suns|33.427083333333336|
|      Boston Celtics|28.544303797468356|
|    Dallas Mavericks| 31.48235294117647|
|New Orleans Pelicans|24.638297872340427|
|       Brooklyn Nets|26.180555555555557|
|     New York Knicks|29.441558441558442|
|Minnesota Timberw...| 28.88235294117647|
|   Memphis Grizzlies|26.634146341463413|
|  Los Angeles Lakers|34.285714285714285|
|Golden State Warr...| 29.38144329896907|
|       Orlando Magic|25.346153846153847|
|Los Angeles Clippers|25.025641025641026|
|     Detroit Pistons| 29.48780487804878|
|       Chicago Bulls| 27.98181818181818|
|     Milwaukee Bucks| 28.46590909090909|
| Cleveland Cavaliers|29.886363636363637|
|      Indiana Pacers|30.468354430379748|
|  Washington Wizards|             30.37|
|       Atlanta Hawks|             31.77|
|           Utah Jazz|31.301204819277107|
|   Charlotte Hornets|            

In [421]:
import pyspark.sql.functions as func


draft_avg = draft_avg \
.withColumn("Pk_avg", func.round(draft_avg["Pk_avg"], 2))



In [422]:
draft_avg.show(300)

+--------------------+------+
|                Team|Pk_avg|
+--------------------+------+
|        Phoenix Suns| 33.43|
|      Boston Celtics| 28.54|
|    Dallas Mavericks| 31.48|
|New Orleans Pelicans| 24.64|
|       Brooklyn Nets| 26.18|
|     New York Knicks| 29.44|
|Minnesota Timberw...| 28.88|
|   Memphis Grizzlies| 26.63|
|  Los Angeles Lakers| 34.29|
|Golden State Warr...| 29.38|
|       Orlando Magic| 25.35|
|Los Angeles Clippers| 25.03|
|     Detroit Pistons| 29.49|
|       Chicago Bulls| 27.98|
|     Milwaukee Bucks| 28.47|
| Cleveland Cavaliers| 29.89|
|      Indiana Pacers| 30.47|
|  Washington Wizards| 30.37|
|       Atlanta Hawks| 31.77|
|           Utah Jazz|  31.3|
|   Charlotte Hornets|  21.5|
|Portland Trail Bl...| 30.49|
|  Philadelphia 76ers|  31.2|
|     Toronto Raptors| 24.21|
|   San Antonio Spurs| 37.14|
|Oklahoma City Thu...| 30.36|
|    Sacramento Kings| 28.34|
|     Houston Rockets| 30.73|
|          Miami Heat| 32.09|
|      Denver Nuggets| 29.65|
+---------

In [423]:
champions= champions.drop('Runner_Ups')

In [424]:
draft.printSchema()

root
 |-- Player: string (nullable = true)
 |-- All_NBA: integer (nullable = true)
 |-- All_Star: integer (nullable = true)
 |-- Draft_Yr: integer (nullable = true)
 |-- Team: string (nullable = true)
 |-- Yrs: integer (nullable = true)
 |-- Games: integer (nullable = true)
 |-- Minutes_Played: integer (nullable = true)
 |-- PTS: integer (nullable = true)
 |-- AST: integer (nullable = true)



In [425]:
champions.show(3000)

+----+------+--------------------+--------------------+--------------------+-----------------+-------------------+
|Year|League|            Champion|           Runner_Up|                Team|Contador_Campeoes|Contador_Runner_Ups|
+----+------+--------------------+--------------------+--------------------+-----------------+-------------------+
|null|  null|                null|                null|        Phoenix Suns|                0|                  3|
|2008|   NBA|      Boston Celtics|  Los Angeles Lakers|      Boston Celtics|               17|                  4|
|1986|   NBA|      Boston Celtics|     Houston Rockets|      Boston Celtics|               17|                  4|
|1984|   NBA|      Boston Celtics|  Los Angeles Lakers|      Boston Celtics|               17|                  4|
|1981|   NBA|      Boston Celtics|     Houston Rockets|      Boston Celtics|               17|                  4|
|1976|   NBA|      Boston Celtics|        Phoenix Suns|      Boston Celtics|    

In [426]:
champions.show(3000)

+----+------+--------------------+--------------------+--------------------+-----------------+-------------------+
|Year|League|            Champion|           Runner_Up|                Team|Contador_Campeoes|Contador_Runner_Ups|
+----+------+--------------------+--------------------+--------------------+-----------------+-------------------+
|null|  null|                null|                null|        Phoenix Suns|                0|                  3|
|2008|   NBA|      Boston Celtics|  Los Angeles Lakers|      Boston Celtics|               17|                  4|
|1986|   NBA|      Boston Celtics|     Houston Rockets|      Boston Celtics|               17|                  4|
|1984|   NBA|      Boston Celtics|  Los Angeles Lakers|      Boston Celtics|               17|                  4|
|1981|   NBA|      Boston Celtics|     Houston Rockets|      Boston Celtics|               17|                  4|
|1976|   NBA|      Boston Celtics|        Phoenix Suns|      Boston Celtics|    

In [427]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, HiveContext
from pyspark.sql import functions as F


champions = champions.withColumn("Contador_Runner_Ups", F.when(F.col("Contador_Runner_Ups").isNull(), 0).otherwise(F.col("Contador_Runner_Ups")))

In [428]:
champions.show(3000)

+----+------+--------------------+--------------------+--------------------+-----------------+-------------------+
|Year|League|            Champion|           Runner_Up|                Team|Contador_Campeoes|Contador_Runner_Ups|
+----+------+--------------------+--------------------+--------------------+-----------------+-------------------+
|null|  null|                null|                null|        Phoenix Suns|                0|                  3|
|2008|   NBA|      Boston Celtics|  Los Angeles Lakers|      Boston Celtics|               17|                  4|
|1986|   NBA|      Boston Celtics|     Houston Rockets|      Boston Celtics|               17|                  4|
|1984|   NBA|      Boston Celtics|  Los Angeles Lakers|      Boston Celtics|               17|                  4|
|1981|   NBA|      Boston Celtics|     Houston Rockets|      Boston Celtics|               17|                  4|
|1976|   NBA|      Boston Celtics|        Phoenix Suns|      Boston Celtics|    

In [429]:
#criação da primeira tabela 
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS NBA LOCATION 'hdfs://hdfs-nn:9000/Projeto/NBA.db'
    """
)
spark.sql(
    """
    DROP TABLE IF EXISTS NBA.draft_gold
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE NBA.draft_gold (
        Player string ,
        All_NBA integer ,
        All_Star integer ,
        Draft_Yr integer ,
        Team string ,
        Yrs integer ,
        Games integer ,
        Minutes_Played integer ,
        PTS integer ,
        AST integer
        
        
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Projeto/NBA.db/draft_gold/'
    """
)

22/01/14 17:17:45 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `nba`.`draft_gold` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [430]:
draft \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Projeto/NBA.db/draft_gold/")

22/01/14 17:17:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [431]:
spark.table("NBA.draft_gold").show(3000)

+--------------------+-------+--------+--------+--------------------+---+-----+--------------+-----+-----+
|              Player|All_NBA|All_Star|Draft_Yr|                Team|Yrs|Games|Minutes_Played|  PTS|  AST|
+--------------------+-------+--------+--------+--------------------+---+-----+--------------+-----+-----+
|       James Johnson|      0|       0|    2009|       Chicago Bulls|  7|  398|          7270| 2594|  590|
|          Taj Gibson|      0|       0|    2009|       Chicago Bulls|  7|  507|         12668| 4641|  504|
|      Kevin Seraphin|      0|       0|    2010|       Chicago Bulls|  6|  374|          5871| 2284|  223|
|         Norris Cole|      0|       0|    2011|       Chicago Bulls|  5|  347|          7893| 2463|  967|
|        Jimmy Butler|      0|       0|    2011|       Chicago Bulls|  5|  323|         10071| 4392|  837|
|         Malcolm Lee|      0|       0|    2011|       Chicago Bulls|  3|   36|           534|  140|   51|
|      Marquis Teague|      0|       

In [432]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Projeto/NBA.db/draft_gold/`
""").show()

++
||
++
++



In [433]:
spark.sql("""
DROP TABLE IF EXISTS NBA.drafts_gold_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS NBA.draft_gold_presto(
        Player string ,
        All_NBA integer ,
        All_Star integer ,
        Draft_Yr integer ,
        Team string ,
        Yrs integer ,
        Games integer ,
        Minutes_Played integer ,
        PTS integer ,
        AST integer
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/Projeto/NBA.db/draft_gold/_symlink_format_manifest/'
""").show()

++
||
++
++

++
||
++
++



In [ ]:
champions.printSchema()

In [434]:
#criação da segunda tabela 
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS NBA LOCATION 'hdfs://hdfs-nn:9000/Projeto/NBA.db'
    """
)
spark.sql(
    """
    DROP TABLE IF EXISTS NBA.champions_gold
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE NBA.champions_gold (
        Year integer ,
        League string ,
        Champion string ,
        Runner_Up string ,
        Team string,
        Contador_Campeoes integer,
        Contador_Runner_Ups integer
        
        
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Projeto/NBA.db/champions_gold/'
    """
)

22/01/14 17:18:16 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `nba`.`champions_gold` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [435]:
champions \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Projeto/NBA.db/champions_gold/")

22/01/14 17:18:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/01/14 17:18:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [436]:
spark.table("NBA.champions_gold").show(3000)

+----+------+--------------------+--------------------+--------------------+-----------------+-------------------+
|Year|League|            Champion|           Runner_Up|                Team|Contador_Campeoes|Contador_Runner_Ups|
+----+------+--------------------+--------------------+--------------------+-----------------+-------------------+
|2020|   NBA|  Los Angeles Lakers|          Miami Heat|  Los Angeles Lakers|               16|                 15|
|2010|   NBA|  Los Angeles Lakers|      Boston Celtics|  Los Angeles Lakers|               16|                 15|
|2009|   NBA|  Los Angeles Lakers|       Orlando Magic|  Los Angeles Lakers|               16|                 15|
|2002|   NBA|  Los Angeles Lakers|       Brooklyn Nets|  Los Angeles Lakers|               16|                 15|
|2001|   NBA|  Los Angeles Lakers|  Philadelphia 76ers|  Los Angeles Lakers|               16|                 15|
|2000|   NBA|  Los Angeles Lakers|      Indiana Pacers|  Los Angeles Lakers|    

In [437]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Projeto/NBA.db/champions_gold/`
""").show()

++
||
++
++



In [438]:
spark.sql("""
DROP TABLE IF EXISTS NBA.champions_gold_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS NBA.champions_gold_presto(
        Year integer ,
        League string ,
        Champion string ,
        Runner_Up string,
        Team string,
        Contador_Campeoes integer,
        Contador_Runner_Ups integer
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/Projeto/NBA.db/champions_gold/_symlink_format_manifest/'
""").show()

22/01/14 17:18:33 WARN DataSource: All paths were ignored:
  hdfs://hdfs-nn:9000/Projeto/NBA.db/champions_gold/_symlink_format_manifest


++
||
++
++

++
||
++
++



In [ ]:
draft_avg.printSchema()

In [439]:
#criação da terceira tabela 
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS NBA LOCATION 'hdfs://hdfs-nn:9000/Projeto/NBA.db'
    """
)
spark.sql(
    """
    DROP TABLE IF EXISTS NBA.draft_avg_gold
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE NBA.draft_avg_gold (
        Team string ,
        Pk_avg double
        
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Projeto/NBA.db/draft_avg_gold/'
    """
)

22/01/14 17:18:54 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `nba`.`draft_avg_gold` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [440]:
draft_avg \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Projeto/NBA.db/draft_avg_gold/")

22/01/14 17:18:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/01/14 17:18:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/01/14 17:18:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/01/14 17:18:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/01/14 17:18:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [441]:
spark.table("NBA.champions_gold").show()

+----+------+--------------------+-------------------+--------------------+-----------------+-------------------+
|Year|League|            Champion|          Runner_Up|                Team|Contador_Campeoes|Contador_Runner_Ups|
+----+------+--------------------+-------------------+--------------------+-----------------+-------------------+
|2020|   NBA|  Los Angeles Lakers|         Miami Heat|  Los Angeles Lakers|               16|                 15|
|2010|   NBA|  Los Angeles Lakers|     Boston Celtics|  Los Angeles Lakers|               16|                 15|
|2009|   NBA|  Los Angeles Lakers|      Orlando Magic|  Los Angeles Lakers|               16|                 15|
|2002|   NBA|  Los Angeles Lakers|      Brooklyn Nets|  Los Angeles Lakers|               16|                 15|
|2001|   NBA|  Los Angeles Lakers| Philadelphia 76ers|  Los Angeles Lakers|               16|                 15|
|2000|   NBA|  Los Angeles Lakers|     Indiana Pacers|  Los Angeles Lakers|             

In [442]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Projeto/NBA.db/draft_avg_gold/`
""").show()

++
||
++
++



In [443]:
spark.sql("""
DROP TABLE IF EXISTS NBA.draft_avg_gold_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS NBA.draft_avg_gold_presto(
        Team string ,
        Pk_avg double
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/Projeto/NBA.db/draft_avg_gold/_symlink_format_manifest/'
""").show()

22/01/14 17:19:04 WARN DataSource: All paths were ignored:
  hdfs://hdfs-nn:9000/Projeto/NBA.db/draft_avg_gold/_symlink_format_manifest


++
||
++
++

++
||
++
++

